In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from rac.pred_models import CustomTensorDataset, ACCNet
import numpy as np
np.random.seed(42)

In [2]:
transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=True,
                                        download=True, transform=transform)
#trainset.data = trainset.data
#trainset.targets = trainset.targets
X_train = trainset.data
y_train = trainset.targets
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1024, num_workers=8)

X_test = testset.data
y_test = testset.targets

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import numpy as np
X_train = np.load("X_train_cifar.npy")
y_train = np.load("Y_train_cifar.npy")
X_test = np.load("X_test_cifar.npy")
y_test = np.load("Y_test_cifar.npy")

In [86]:
#X = np.load("../datasets/cifar10_data/cifar10_embedding.npy")
#Y = np.load("../datasets/cifar10_data/cifar10_labels.npy")

In [87]:
#from sklearn import preprocessing
#X = preprocessing.StandardScaler().fit_transform(X)
#from sklearn.decomposition import PCA
#pca = PCA(n_components=100)
#X = pca.fit_transform(X)

In [88]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [89]:
#X.shape

In [4]:
def clustering_from_clustering_solution(clustering_solution):
    num_clusters = np.max(clustering_solution) + 1
    clustering = [[] for _ in range(num_clusters)]
    for i in range(len(clustering_solution)):
        clustering[clustering_solution[i]].append(i)
    return clustering, num_clusters

def sim_matrix_from_clustering(clustering, N):
    pairwise_similarities = -np.ones((N, N))
    for cind in clustering:
        pairwise_similarities[np.ix_(cind, cind)] = 1
    return pairwise_similarities

In [5]:
train_sol = clustering_from_clustering_solution(y_train[:10000])
train_sim_matrix = sim_matrix_from_clustering(train_sol[0], len(y_train[:10000]))

test_sol = clustering_from_clustering_solution(y_test[:10000])
test_sim_matrix = sim_matrix_from_clustering(test_sol[0], len(y_test[:10000]))

In [6]:
train_sim_matrix.shape

(10000, 10000)

In [7]:
def get_pairs(prop_pos, prop_neg, sim_matrix, data):
    N = sim_matrix.shape[0]
    lower_triangle_indices = np.tril_indices(N, -1)
    ind_pos = np.where(sim_matrix[lower_triangle_indices] == 1)[0]
    ind_neg = np.where(sim_matrix[lower_triangle_indices] == -1)[0]
    num_pos = int(len(ind_pos)*prop_pos)
    num_neg = int(len(ind_neg)*prop_neg)
    print("num_pos: ", num_pos)
    print("num_neg: ", num_neg)
    ind_pos = np.random.choice(ind_pos, num_pos)
    ind_neg = np.random.choice(ind_neg, num_neg)
    if num_pos < num_neg:
        indices = np.concatenate([ind_neg, ind_pos])
    else:
        indices = np.concatenate([ind_pos, ind_neg])
    ind1, ind2 = lower_triangle_indices[0][indices], lower_triangle_indices[1][indices]
    x1 = data[ind1]
    x2 = data[ind2]
    y = sim_matrix[ind1, ind2]
    lab1 = np.where(y >= 0)
    lab2 = np.where(y < 0)
    y[lab1] = 1.0
    y[lab2] = 0.0
    return x1, x2, y

In [9]:
x1_train, x2_train, y_train_pairs = get_pairs(0.001, 0.0004, train_sim_matrix, X_train)
x1_test, x2_test, y_test_pairs = get_pairs(0.001, 0.0001, test_sim_matrix, X_test)

num_pos:  4996
num_neg:  17999
num_pos:  4995
num_neg:  4500


In [9]:
cifar_training_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar_test_transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = CustomTensorDataset(x1_train, x2_train, y_train_pairs, train=True, transform=cifar_training_transform)
test_dataset = CustomTensorDataset(x1_test, x2_test, y_test_pairs, transform=cifar_test_transform)  

In [10]:
train_dataset = CustomTensorDataset(torch.Tensor(x1_train), torch.Tensor(x2_train), torch.Tensor(y_train_pairs), train=True, transform=None)
test_dataset = CustomTensorDataset(torch.Tensor(x1_test), torch.Tensor(x2_test), torch.Tensor(y_test_pairs), transform=None)

In [11]:
batch_size = 128
#class_sample_count = [10, 1, 20, 3, 4] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
class_sample_count = np.unique(y_train_pairs, return_counts=True)[1].tolist()
#print(class_sample_count)
weights = 1/torch.Tensor(class_sample_count)
weights = weights[y_train_pairs]
#print(weights)
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1024)
#trainloader = data_utils.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, sampler = sampler)

In [12]:
#for x1, x2, y in train_loader:
    #print(np.unique(y, return_counts=True))

In [13]:
len(train_dataset)

35998

In [14]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
device

device(type='cuda', index=0)

In [16]:
net = ACCNet(base_net="three_layer_net", siamese=True, input_dim=512).to(device)

In [17]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=5e-4)
print("training...")
print(len(train_dataset))
for epoch in range(150):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    step = 0
    for i, data in enumerate(train_loader, 0):
        #print(np.unique(data[2], return_counts=True))
        #print(data[2])
        # get the inputs; data is a list of [inputs, labels]
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(x1, x2)
        outputs = outputs.reshape((outputs.shape[0]))
        #labels = labels.reshape((labels.shape[0], 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        step += 1
    print("loss: ", running_loss/step)
    step = 0
    running_loss = 0.0

training...
35998
0
loss:  0.27280678057131613
1
loss:  0.04018638282518913
2
loss:  0.018802124435570857
3
loss:  0.011270246141737142
4
loss:  0.007788570391976915
5
loss:  0.00667369461528369
6
loss:  0.003019441327465911
7
loss:  0.0031338355776876358
8
loss:  0.0026594920304016845
9
loss:  0.0002779298639842772
10
loss:  0.0001228516807939947
11
loss:  5.746927079321754e-05
12
loss:  4.203114307801702e-05
13
loss:  3.2130345831490505e-05
14
loss:  2.6048067858251277e-05
15
loss:  2.18244759649909e-05
16
loss:  1.7593279361538774e-05
17
loss:  1.4529788384136072e-05
18
loss:  1.2143002431823772e-05
19
loss:  1.0069644046053547e-05
20
loss:  8.494534593925025e-06
21
loss:  7.36364170612656e-06
22
loss:  6.078499926906397e-06
23
loss:  4.987663891430042e-06
24
loss:  4.216642383759809e-06
25
loss:  3.620891990036682e-06
26
loss:  2.9944599567583163e-06
27
loss:  2.6346067505841523e-06
28
loss:  2.1429335418400617e-06
29
loss:  1.7942270459850445e-06
30
loss:  1.5529337161638976e-06
3

In [18]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
preds = []
with torch.no_grad():
    for data in test_loader:
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # calculate outputs by running images through the network
        outputs = net(x1, x2)
        pred = nn.Sigmoid()(outputs)
        #print(pred)
        pred = pred.cpu().numpy()
        pred = pred.reshape(pred.shape[0])
        pred[pred >= 0.5] = 1
        pred[pred < 0.5] = 0
        #print(torch.max(pred.data, 1)[1])
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(pred.data, 1)
        preds.extend(pred.tolist())


In [19]:
len(preds)

9495

In [20]:
np.unique(preds, return_counts=True)

(array([0., 1.]), array([5406, 4089]))

In [21]:
#preds

In [22]:
len(y_test_pairs)


9495

In [23]:
y_test_pairs

array([1., 1., 1., ..., 0., 0., 0.])

In [24]:
np.unique(y_test_pairs, return_counts=True)

(array([0., 1.]), array([4500, 4995]))

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test_pairs, preds))

              precision    recall  f1-score   support

         0.0       0.82      0.98      0.89      4500
         1.0       0.98      0.80      0.88      4995

    accuracy                           0.89      9495
   macro avg       0.90      0.89      0.89      9495
weighted avg       0.90      0.89      0.89      9495



In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test_pairs, preds))

              precision    recall  f1-score   support

         0.0       0.81      0.98      0.88      4500
         1.0       0.97      0.79      0.87      4995

    accuracy                           0.88      9495
   macro avg       0.89      0.88      0.88      9495
weighted avg       0.89      0.88      0.88      9495

